In [69]:
import requests
import pandas as pd

In [70]:
#input video url
#referer = input("Enter referer: ")

#test url
referer = 'https://www.tiktok.com/@toanchodien23/video/7121230509027478810'


In [ ]:
cookies = {
    '_ttp': '29R1dUogCz9evh5bjMrLFIxQJ3L',
    's_v_web_id': 'verify_l4ct1juv_06cHWVBK_PaGW_4Daa_9ZKq_NoNxlIx4BD2W',
    'tt_csrf_token': 'iztXcztc-odZDYbduwcZE8q_Pr04KVA0Iq_0',
    'csrf_session_id': '5ceea1925aa09df79cacdf7da670460b',
    'passport_csrf_token': '94175df818e9485d08df6a1fcd70ea6b',
    'passport_csrf_token_default': '94175df818e9485d08df6a1fcd70ea6b',
    'd_ticket': '1f4b47f14971d6f2d9dab12eac466dea9cb6b',
    'passport_auth_status': '413b8fdc2e94511cd587f963e312de3a%2C',
    'passport_auth_status_ss': '413b8fdc2e94511cd587f963e312de3a%2C',
    'sid_guard': '076bd311dc7a4f00da0f5cf854ec9931%7C1661858152%7C5184000%7CSat%2C+29-Oct-2022+11%3A15%3A52+GMT',
    'uid_tt': '609bfb461acb534d23bbc9c8c2e05679c64930e0152a1a521ccac0e72e0d1970',
    'uid_tt_ss': '609bfb461acb534d23bbc9c8c2e05679c64930e0152a1a521ccac0e72e0d1970',
    'sid_tt': '076bd311dc7a4f00da0f5cf854ec9931',
    'sessionid': '076bd311dc7a4f00da0f5cf854ec9931',
    'sessionid_ss': '076bd311dc7a4f00da0f5cf854ec9931',
    'sid_ucp_v1': '1.0.0-KDhlZTc5MWFkNTFhN2Q4ZmVhNTZjZmE5OTcyY2E5MzQyNjIwZDczYjAKIAiviMHE15jl_2EQ6OK3mAYYswsgDDDpuf6PBjgCQPEHEAMaBm1hbGl2YSIgMDc2YmQzMTFkYzdhNGYwMGRhMGY1Y2Y4NTRlYzk5MzE',
    'ssid_ucp_v1': '1.0.0-KDhlZTc5MWFkNTFhN2Q4ZmVhNTZjZmE5OTcyY2E5MzQyNjIwZDczYjAKIAiviMHE15jl_2EQ6OK3mAYYswsgDDDpuf6PBjgCQPEHEAMaBm1hbGl2YSIgMDc2YmQzMTFkYzdhNGYwMGRhMGY1Y2Y4NTRlYzk5MzE',
    'store-idc': 'useast2a',
    'store-country-code': 'vn',
    'store-country-code-src': 'uid',
    'tt-target-idc': 'alisg',
    'cmpl_token': 'AgQQAPOFF-RMpbHBemLwop0W-MbaVVbI_4AOYMU5HQ',
    '__tea_cache_tokens_1988': '{%22user_unique_id%22:%227108724646376621570%22%2C%22timestamp%22:1661858807046%2C%22_type_%22:%22default%22}',
    '_abck': '8197FC42F83990B3157D81C84D21BFD5~-1~YAAQLlBNGy+pFP2CAQAApc0yEwh5THB5lqykWDfDuOSnDhLENnWneOCctfKK79MQKLhRkfcyyL6FkvTeoe5oheL9BB5Wnm0DtkbRhfsxKXEp6CrT59JKEdG7foqhO2tHOoDjIxFxR/eD4VCSGAH/ZsYt4nKeLPukfUSr8qo3a718HD6LEl2LqJ/AQVcbPTqDc7yIP/t+8l6RtzON/VWGPCk2kjEDkkuIfCR9iuxSB6VbQiyzGY64f0PN57pTO06f57DhCWkWrsRbDaEJU13q8KnCY+SIGpWzNjBHLe5hgJkHFPZ61c5gLg9POWo8h74C9zLHyCFjO00lYNxXKREokNJ2jBcI3ayep3YbBnXQKikltoEiaVv66AUu75t4hIp+F0mSonYueAoLnF4=~-1~-1~-1',
    'bm_sz': '404AF1D62F9E243D5F6FCD16F1D79D33~YAAQLlBNGzCpFP2CAQAApc0yExElRwaYOIcXoiKgwpmRjn9dKEWmsDY2Dm27YQhFAC5ishW/4ypvHZTW6E0461o3nBK9aLv88rKHnkPzLjX5bJGykjtUQJemguJ2EH3RHNyGZNsfFeUrZrjUCB5v13qfJbQxqXd7YSxpbbMMDO6raS5Vf0amMXJXVLX0NfEXl3PuYmmYMDCy1ux/tPy2A2Y5CympMcMyeH4eK2TZ0yF5PkhG9C/YPPjyHvQNOHrjRvXgPseYlMQ2V+H3U3BF4g+CYKU5Zftzw3CfyZPddjTZHU8=~3291184~3552069',
    'ak_bmsc': '2870BDC16E4584CF994EC4C65A327156~000000000000000000000000000000~YAAQLlBNG1WpFP2CAQAAGPsyExHF/yR1s4mRHxBNig7s+Y/e5JqtPdJ9tAaYPlPrbyEMaFptEnx2jGPtwWiiMGUOaS8i+4oh7iQ2xG52DDihaYxPTM+SgAbk6ews7wcqRRUiHibRy0m4bE/HNLBdeXM66iN/M/46XkoA7PwLjuNYwWYAVn3/zidWyE/Vl8RiqEElDCGxxm9YsQ70O6rBUX2dXWOXwMtYnH5jVL8sWmTF0yRUmpfla3If6m+3R/eRzyzNO/qc2Wt5laZC6nS7xCCQdQdIOd2SF/f4efS4NcXJwk2Q25+cLND2VDq6hNi1d9X7U+j35tYGU8ZCYuH0SbcNu8tVOVlvTSCpsCi0qDpP/iNo9VI2DHmh/xTR0JHfwZXFfG0Oa4W9VfSG',
    'ttwid': '1%7CqDEYDh1xjNzhQymfMQXUBbiZ0avlnad6c477HMaAjz4%7C1662474926%7C9ac339ad183588a6710e9ff9e338c8b891cd14f0179bd579c5b7397f51eaece7',
    'bm_sv': '545CEA156158F5926E3294D27CFFB99A~YAAQHlBNG+g9EP2CAQAArNdCExGvP4b6UtG260o4R1qybCJRXt3Mt+93/f7AprmvInFBjj+5pPUWfS45FHT0LMGmhSwaeAHWIn1+DNGCof10Bx3ZuqzgYQb+MPoSxa0kaMq/pQisAZRSkMY9+40MokluAriz2KVsnzLNrkzUp15EigyB0b9JeH6kAFslDuWwLIkqznDsUDHKRnoviOeW1IYhAR3S4LJxRWO1HfdMQ9GloRwyh3OObmtcYJAtVcYqwA==~1',
    'odin_tt': '1cb27cbeb3f6042a9a1899c374811d708630d0ba434143beec52c62294b46376d428d7a9130836098c93371be779f90c68f8a03ab6d4b96964ae09e45eaa713e',
    'msToken': 'wEZfUhNWpniGXRo_xBGKcmno5MKHejayplzCiD9uifvSZ2-Du763DjS1q8iyR03uUlrAWwwlWDci_opptAVa7acQr165fNciR2A2pxcnyT67APgyYHAW-GJ6AOT3j60yJ7ISYCk=',
    'msToken': 'wEZfUhNWpniGXRo_xBGKcmno5MKHejayplzCiD9uifvSZ2-Du763DjS1q8iyR03uUlrAWwwlWDci_opptAVa7acQr165fNciR2A2pxcnyT67APgyYHAW-GJ6AOT3j60yJ7ISYCk=',
    'passport_fe_beating_status': 'true',
}

headers = {
    'authority': 'www.tiktok.com',
    'accept': '*/*',
    'accept-language': 'en,vi;q=0.9',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_ttp=29R1dUogCz9evh5bjMrLFIxQJ3L; s_v_web_id=verify_l4ct1juv_06cHWVBK_PaGW_4Daa_9ZKq_NoNxlIx4BD2W; tt_csrf_token=iztXcztc-odZDYbduwcZE8q_Pr04KVA0Iq_0; csrf_session_id=5ceea1925aa09df79cacdf7da670460b; passport_csrf_token=94175df818e9485d08df6a1fcd70ea6b; passport_csrf_token_default=94175df818e9485d08df6a1fcd70ea6b; d_ticket=1f4b47f14971d6f2d9dab12eac466dea9cb6b; passport_auth_status=413b8fdc2e94511cd587f963e312de3a%2C; passport_auth_status_ss=413b8fdc2e94511cd587f963e312de3a%2C; sid_guard=076bd311dc7a4f00da0f5cf854ec9931%7C1661858152%7C5184000%7CSat%2C+29-Oct-2022+11%3A15%3A52+GMT; uid_tt=609bfb461acb534d23bbc9c8c2e05679c64930e0152a1a521ccac0e72e0d1970; uid_tt_ss=609bfb461acb534d23bbc9c8c2e05679c64930e0152a1a521ccac0e72e0d1970; sid_tt=076bd311dc7a4f00da0f5cf854ec9931; sessionid=076bd311dc7a4f00da0f5cf854ec9931; sessionid_ss=076bd311dc7a4f00da0f5cf854ec9931; sid_ucp_v1=1.0.0-KDhlZTc5MWFkNTFhN2Q4ZmVhNTZjZmE5OTcyY2E5MzQyNjIwZDczYjAKIAiviMHE15jl_2EQ6OK3mAYYswsgDDDpuf6PBjgCQPEHEAMaBm1hbGl2YSIgMDc2YmQzMTFkYzdhNGYwMGRhMGY1Y2Y4NTRlYzk5MzE; ssid_ucp_v1=1.0.0-KDhlZTc5MWFkNTFhN2Q4ZmVhNTZjZmE5OTcyY2E5MzQyNjIwZDczYjAKIAiviMHE15jl_2EQ6OK3mAYYswsgDDDpuf6PBjgCQPEHEAMaBm1hbGl2YSIgMDc2YmQzMTFkYzdhNGYwMGRhMGY1Y2Y4NTRlYzk5MzE; store-idc=useast2a; store-country-code=vn; store-country-code-src=uid; tt-target-idc=alisg; cmpl_token=AgQQAPOFF-RMpbHBemLwop0W-MbaVVbI_4AOYMU5HQ; __tea_cache_tokens_1988={%22user_unique_id%22:%227108724646376621570%22%2C%22timestamp%22:1661858807046%2C%22_type_%22:%22default%22}; _abck=8197FC42F83990B3157D81C84D21BFD5~-1~YAAQLlBNGy+pFP2CAQAApc0yEwh5THB5lqykWDfDuOSnDhLENnWneOCctfKK79MQKLhRkfcyyL6FkvTeoe5oheL9BB5Wnm0DtkbRhfsxKXEp6CrT59JKEdG7foqhO2tHOoDjIxFxR/eD4VCSGAH/ZsYt4nKeLPukfUSr8qo3a718HD6LEl2LqJ/AQVcbPTqDc7yIP/t+8l6RtzON/VWGPCk2kjEDkkuIfCR9iuxSB6VbQiyzGY64f0PN57pTO06f57DhCWkWrsRbDaEJU13q8KnCY+SIGpWzNjBHLe5hgJkHFPZ61c5gLg9POWo8h74C9zLHyCFjO00lYNxXKREokNJ2jBcI3ayep3YbBnXQKikltoEiaVv66AUu75t4hIp+F0mSonYueAoLnF4=~-1~-1~-1; bm_sz=404AF1D62F9E243D5F6FCD16F1D79D33~YAAQLlBNGzCpFP2CAQAApc0yExElRwaYOIcXoiKgwpmRjn9dKEWmsDY2Dm27YQhFAC5ishW/4ypvHZTW6E0461o3nBK9aLv88rKHnkPzLjX5bJGykjtUQJemguJ2EH3RHNyGZNsfFeUrZrjUCB5v13qfJbQxqXd7YSxpbbMMDO6raS5Vf0amMXJXVLX0NfEXl3PuYmmYMDCy1ux/tPy2A2Y5CympMcMyeH4eK2TZ0yF5PkhG9C/YPPjyHvQNOHrjRvXgPseYlMQ2V+H3U3BF4g+CYKU5Zftzw3CfyZPddjTZHU8=~3291184~3552069; ak_bmsc=2870BDC16E4584CF994EC4C65A327156~000000000000000000000000000000~YAAQLlBNG1WpFP2CAQAAGPsyExHF/yR1s4mRHxBNig7s+Y/e5JqtPdJ9tAaYPlPrbyEMaFptEnx2jGPtwWiiMGUOaS8i+4oh7iQ2xG52DDihaYxPTM+SgAbk6ews7wcqRRUiHibRy0m4bE/HNLBdeXM66iN/M/46XkoA7PwLjuNYwWYAVn3/zidWyE/Vl8RiqEElDCGxxm9YsQ70O6rBUX2dXWOXwMtYnH5jVL8sWmTF0yRUmpfla3If6m+3R/eRzyzNO/qc2Wt5laZC6nS7xCCQdQdIOd2SF/f4efS4NcXJwk2Q25+cLND2VDq6hNi1d9X7U+j35tYGU8ZCYuH0SbcNu8tVOVlvTSCpsCi0qDpP/iNo9VI2DHmh/xTR0JHfwZXFfG0Oa4W9VfSG; ttwid=1%7CqDEYDh1xjNzhQymfMQXUBbiZ0avlnad6c477HMaAjz4%7C1662474926%7C9ac339ad183588a6710e9ff9e338c8b891cd14f0179bd579c5b7397f51eaece7; bm_sv=545CEA156158F5926E3294D27CFFB99A~YAAQHlBNG+g9EP2CAQAArNdCExGvP4b6UtG260o4R1qybCJRXt3Mt+93/f7AprmvInFBjj+5pPUWfS45FHT0LMGmhSwaeAHWIn1+DNGCof10Bx3ZuqzgYQb+MPoSxa0kaMq/pQisAZRSkMY9+40MokluAriz2KVsnzLNrkzUp15EigyB0b9JeH6kAFslDuWwLIkqznDsUDHKRnoviOeW1IYhAR3S4LJxRWO1HfdMQ9GloRwyh3OObmtcYJAtVcYqwA==~1; odin_tt=1cb27cbeb3f6042a9a1899c374811d708630d0ba434143beec52c62294b46376d428d7a9130836098c93371be779f90c68f8a03ab6d4b96964ae09e45eaa713e; msToken=wEZfUhNWpniGXRo_xBGKcmno5MKHejayplzCiD9uifvSZ2-Du763DjS1q8iyR03uUlrAWwwlWDci_opptAVa7acQr165fNciR2A2pxcnyT67APgyYHAW-GJ6AOT3j60yJ7ISYCk=; msToken=wEZfUhNWpniGXRo_xBGKcmno5MKHejayplzCiD9uifvSZ2-Du763DjS1q8iyR03uUlrAWwwlWDci_opptAVa7acQr165fNciR2A2pxcnyT67APgyYHAW-GJ6AOT3j60yJ7ISYCk=; passport_fe_beating_status=true',
    
    'referer': referer,
    # 'sec-ch-ua': '"Microsoft Edge";v="105", " Not;A Brand";v="99", "Chromium";v="105"',
    # 'sec-ch-ua-mobile': '?0',
    # 'sec-ch-ua-platform': '"Windows"',
    # 'sec-fetch-dest': 'empty',
    # 'sec-fetch-mode': 'cors',
    # 'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27',
}

# originURL = 'https://www.tiktok.com/api/comment/list/?aid=1988&app_language=ja-JP&app_name=tiktok_web&aweme_id=7121230509027478810&battery_info=1&browser_language=en&browser_name=Mozilla&browser_online=true&browser_platform=Win32&browser_version=5.0%20%28Windows%20NT%2010.0%3B%20Win64%3B%20x64%29%20AppleWebKit%2F537.36%20%28KHTML%2C%20like%20Gecko%29%20Chrome%2F105.0.0.0%20Safari%2F537.36%20Edg%2F105.0.1343.27&channel=tiktok_web&cookie_enabled=true&count=20&current_region=JP&cursor=20&device_id=7108724646376621570&device_platform=web_pc&focus_state=true&fromWeb=1&from_page=video&history_len=10&is_fullscreen=false&is_page_visible=true&os=windows&priority_region=VN&referer=&region=VN&screen_height=1080&screen_width=1920&tz_name=Asia%2FBangkok&verifyFp=verify_l4ct1juv_06cHWVBK_PaGW_4Daa_9ZKq_NoNxlIx4BD2W&webcast_language=en&msToken=wEZfUhNWpniGXRo_xBGKcmno5MKHejayplzCiD9uifvSZ2-Du763DjS1q8iyR03uUlrAWwwlWDci_opptAVa7acQr165fNciR2A2pxcnyT67APgyYHAW-GJ6AOT3j60yJ7ISYCk=&X-Bogus=DFSzswVuG6sANyEZSQ/bHQYklTIs&_signature=_02B4Z6wo00001ZmjMMwAAIDAR9Voaj4U-lGZozRAAAWAfc'

#cursor = 20
#count = 20
#response = requests.get('https://www.tiktok.com/api/comment/list/?aid=1988&app_language=ja-JP&app_name=tiktok_web&aweme_id=7121230509027478810&cookie_enabled=true&count=20&current_region=JP&cursor=20&device_id=7108724646376621570&device_platform=web_pc&focus_state=true&fromWeb=1&from_page=video&history_len=10&is_fullscreen=false&is_page_visible=true&os=windows&priority_region=VN&referer=&region=VN&screen_height=1080&screen_width=1920&tz_name=Asia%2FBangkok&verifyFp=verify_l4ct1juv_06cHWVBK_PaGW_4Daa_9ZKq_NoNxlIx4BD2W&webcast_language=en&msToken=wEZfUhNWpniGXRo_xBGKcmno5MKHejayplzCiD9uifvSZ2-Du763DjS1q8iyR03uUlrAWwwlWDci_opptAVa7acQr165fNciR2A2pxcnyT67APgyYHAW-GJ6AOT3j60yJ7ISYCk=&X-Bogus=DFSzswVuG6sANyEZSQ/bHQYklTIs&_signature=_02B4Z6wo00001ZmjMMwAAIDAR9Voaj4U-lGZozRAAAWAfc', cookies=cookies, headers=headers)

#cursor = 0
#count = max-50
response = requests.get('https://www.tiktok.com/api/comment/list/?aweme_id=7121230509027478810&count=50&cursor=4500', headers=headers)

json_data = response.json()
print(len(json_data['comments']))
count = 0
for data in json_data['comments']:
    print(f"{count}. \tNickname: {data['user']['nickname']} \n\t Comment: {data['text']}\n")
    count += 1

In [ ]:
#Get the next cursor - next 50 comment (if there is any)


In [ ]:
json_data = response.json()
count = 0
for data in json_data['comments']:
    print(f"{count}. \tNickname: {data['user']['nickname']} \n\t Comment: {data['text']}\n")
    count += 1